In [162]:
#packages
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.linalg import qr
from scipy.sparse import csr_matrix

In [163]:
#functions
def sketch_matrix(m, n_columns, non_zero_entries):
    #matrix with all zero entries
    S = np.zeros((m, n_columns))
    scaling_factor = 1 / np.sqrt(non_zero_entries)
    #loop through each columns to edit the non zero entries in
    for col in range(n_columns):
        # Randomly select position of non_zero entries
        nz_positions = np.random.choice(m, non_zero_entries, replace=False)
        
        # Randomly assign values of either 1 or -1 to these positions
        values = np.random.choice([1, -1], non_zero_entries)* scaling_factor
        
        # Assign the values to the selected positions in the column
        for idx, value in zip(nz_positions, values):
            S[idx, col] = value
    
    return S

#Higher leverage scores indicate more influential data points.
def estimate_leverage_scores(A, R, gamma):
    """Estimate leverage scores ˜li for each row using matrix R (similar to Lemma 5.1)."""
    n, d = A.shape
    k = int(np.ceil(d / gamma))  # Choose k based on γ
    G = np.random.randn(d, k) / np.sqrt(k) #scale the matrix
    
    # Compute the leverage scores ˜li = || e_i^T AR G ||_2^2
    ARG = A @ (R @ G)
    leverage_scores = np.sum(ARG ** 2, axis=1)  # || e_i^T AR G ||_2^2
    
    return leverage_scores

def fast_least_squares_sgd(A, b, non_zero_entries,T=100, eta=0.001, gamma=0.1, batch_size=10):
    """
    Fast least squares via preconditioned mini-batch SGD using sketch matrix and leverage scores.
    
    Parameters:
    - A: (n, d) NumPy array (or sparse matrix), the design matrix.
    - b: (n,) NumPy array, the target vector.
    - T: Number of SGD iterations.
    - eta: Learning rate.
    - gamma: Approximation parameter for leverage score estimation.
    - batch_size: Number of rows sampled per SGD iteration.
    
    Returns:
    - x: (d,) NumPy array, the estimated least squares solution.
    """
    n, d = A.shape
    m = min(2 * d, n)  # Typically 2d rows for sketch matrix
    
    # Step 1: Generate the sketch matrix S using the sketch_matrix function
    S = sketch_matrix(m, n, non_zero_entries)

    # Step 2: Compute SA and Sb
    SA = S @ A
    Sb = S @ b

    # Step 3: Compute QR decomposition of SA
    Q, R_inv = np.linalg.qr(SA)
    R = np.linalg.inv(R_inv)

    # Step 4: Compute leverage score estimates
    leverage_scores = estimate_leverage_scores(A, R, gamma)
    leverage_probs = leverage_scores / np.sum(leverage_scores)  # Normalize

    # Step 5: Compute initial x0 by solving (SAx = Sb)
    x = np.linalg.lstsq(SA, Sb, rcond=None)[0]

    # Step 6: Perform mini-batch SGD
    for t in range(T):
        # Step 6.1: Sample batch indices based on leverage scores
        batch_indices = np.random.choice(n, size=batch_size, p=leverage_probs)
        
        # Step 6.2: Construct StA and Stb for mini-batch
        StA = A[batch_indices]
        Stb = b[batch_indices]

        # Step 6.3: Compute gradient of the least squares loss
        gt = 2 * StA.T @ (StA @ x - Stb)

        # Step 6.4: Update x using preconditioned gradient
        x = x - eta * R @ (R.T @ gt)

    return x

In [164]:
sketch_matrix(5,3,1)

array([[ 0.,  0.,  0.],
       [ 0., -1.,  1.],
       [ 0.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [165]:
n, d = 10000, 100 
non_zero_entries = 10
A = np.random.randn(n, d)  # Design matrix A
x_true = np.random.randn(d)  # True solution vector x
b = A @ x_true + np.random.randn(n) * 0.1


# Estimate the solution using the algorithm
x_estimated = fast_least_squares_sgd(A, b, 10)


In [166]:
mse = np.mean((x_estimated - x_true) ** 2)
mse

0.00011161879635956588

In [167]:
# Solve least squares using NumPy's lstsq (normal OLS solution)
x_ols = np.linalg.lstsq(A, b, rcond=None)[0]

# Calculate the MSE for OLS
mse_ols = np.mean((x_ols - x_true) ** 2)
mse_ols

1.0946171174895882e-06